# Logistic Regression Algorithm

## Classical imports and getting the dataset

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Getting the clean datasets :

In [ ]:
%run /cleanDataV2.ipynb

Checking that the datasets are ok :

In [ ]:
tf #the training dataset

In [ ]:
rf #the testing dataset

In [ ]:
#Deleting the Id (we don't want our training to take into account the Id)
tf=tf.drop(columns='userId')

## Logistic Regression Model

In [ ]:
#To see what the best parameters are
def best_model(model):
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)

In [ ]:
#Splitting our dataset into one set of training and one set of test
y = tf.labels.values
x = tf.drop(['labels'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [ ]:
#Logistic Regression
param_grid = {'C': [0.1,0.2,0.5,1,10], 'max_iter': [250, 500], 'fit_intercept':[True],'intercept_scaling':[1],
              'penalty':['l2'], 'tol':[0.00001,0.0001,0.000001]} #testing different parameters for C, max_iter and tol

#GridSearch automatically takes the best parameter after testing all the options
lr = GridSearchCV(LogisticRegression(solver='lbfgs'),param_grid, cv=5, refit=True, verbose=0) 
lr.fit(x_train,y_train) #the fitting

acc = lr.score(x_test,y_test)*100 #calculating the accuracy

print(best_model(lr)) # to show the best parameters found by GridSearch
print("Test Accuracy {:.2f}%".format(acc)) # to show the accuracy

## Prediction

In [ ]:
#Predicting, and exporting to csv to make a submission
sol=pd.DataFrame({'userId':rf['userId']}) #creating the dataframe for the solution and getting the ID
rf=rf.drop(columns='userId') #dropping the id to have the same structure as the training dataset

y_pred = lr.predict(rf) #the prediction
sol['washDishes'] = y_pred
sol["washDishes"] = sol["washDishes"].apply(lambda e: 'Auto' if e == 1 else 'Hand') #replacing the -1 and 1 with the original values
sol.to_csv('./DS_CentraleSupelec_ST42021/lrsolution.csv', index=False) #exporting to csv without the index
sol #checking the output